In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import Simulator
from elfpy.utils.post_processing import analysis_dict_to_dataframe
from elfpy.utils import sim_utils  # utilities for setting up a simulation
import elfpy.utils.parse_config as config_utils
import elfpy.utils.outputs as output_utils

In [2]:
pd.set_option("float_format",'{:,.3f}'.format)

In [3]:
start_time = time.time()

config_file = "../../config/example_config.toml"
override_dict = {
    "pricing_model_name": "YieldSpace",
}
# get config & logging level
config = sim_utils.override_config_variables(config_utils.load_and_parse_config_file(config_file), override_dict)
# define root logging parameters
output_utils.setup_logging(
    log_filename="../../.logging/sim_one_model.log", log_level=config.simulator.logging_level
)
# instantiate random number generator
rng = np.random.default_rng(config.simulator.random_seed)
# run random number generators to get random simulation arguments
random_sim_vars = sim_utils.get_random_variables(config, rng)
# instantiate the pricing model
sim_pricing_model = sim_utils.get_pricing_model(model_name=config.amm.pricing_model_name)
# instantiate the market
sim_market = sim_utils.get_market(
    sim_pricing_model,
    random_sim_vars.target_pool_apy,
    random_sim_vars.fee_percent,
    config.simulator.token_duration,
    random_sim_vars.init_share_price,
)
# instantiate the init_lp agent
init_agents = {
    0: sim_utils.get_init_lp_agent(
        config,
        sim_market,
        sim_pricing_model,
        random_sim_vars.target_liquidity,
        random_sim_vars.target_pool_apy,
        random_sim_vars.fee_percent,
    )
}
# set up simulator with only the init_lp_agent
simulator = Simulator(
    config=config,
    pricing_model=sim_pricing_model,
    market=sim_market,
    agents=init_agents,
    rng=rng,
    random_simulation_variables=random_sim_vars,
)
# initialize the market using the LP agent
simulator.collect_and_execute_trades()

num_runs = 1 # lots of runs to have high statistical power
for run_number in range(num_runs): # run a simulation per pricing model
    override_dict = {
        "pricing_model_name": "YieldSpace",
        "num_trading_days": 3,
        "num_blocks_per_day": 5,
        #"vault_apy": [0.1,]*config["num_trading_days"],
        #"init_vault_age": 1,
        #"target_pool_apy" : 0.05,
        #"target_liquidity": 3e6,
        #"fee_percent" : 0.1,
    }
    simulator.random_variables = sim_utils.get_random_variables(config, rng) # reset random variables
    simulator.config = sim_utils.override_config_variables(config, override_dict)
    simulator.run_simulation()

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

Total time for 1 runs was 0.012 seconds; which is 0.012 seconds per run


In [5]:
trades = analysis_dict_to_dataframe(simulator.analysis_dict)

In [7]:
print(f'Simulator contains {trades.shape[1]} variables tracked over {trades.shape[0]} simulations.')
print(f'Trades dataframe variables:\n'+'\t'.join(list(trades.columns)))
print(f'\nTrades dataframe:')
display(pd.concat([trades.head(4), trades.tail(2)]).T)

Simulator contains 40 variables tracked over 5 simulations.
Trades dataframe variables:
index	model_name	run_number	simulation_start_time	day	block_number	daily_block_number	block_timestamp	current_market_datetime	current_market_yearfrac	run_trade_number	market_step_size	position_duration	target_liquidity	fee_percent	floor_fee	init_vault_age	base_asset_price	vault_apy	pool_apy	share_reserves	bond_reserves	base_buffer	bond_buffer	lp_reserves	share_price	init_share_price	num_trading_days	num_blocks_per_day	spot_price	pool_apy_percent	vault_apy_percent	total_liquidity_usd	price_total_return	price_total_return_percent	price_total_return_scaled_to_share_price	share_price_total_return	share_price_total_return_percent	price_total_return_percent_annualized	share_price_total_return_percent_annualized

Trades dataframe:


,0,1,2,3,3,4
index,0,1,2,3,3,4
model_name,YieldSpace,YieldSpace,YieldSpace,YieldSpace,YieldSpace,YieldSpace
run_number,0,0,0,0,0,0
simulation_start_time,NaT,NaT,NaT,2022-12-22 17:52:30.981016+00:00,2022-12-22 17:52:30.981016+00:00,2022-12-22 17:52:30.981016+00:00
day,0,0,0,2,2,2
block_number,0,0,0,14,14,14
daily_block_number,0,0,0,4,4,4
block_timestamp,None,None,None,2022-12-22 17:55:18.981016+00:00,2022-12-22 17:55:18.981016+00:00,2022-12-22 17:55:18.981016+00:00
current_market_datetime,None,None,None,2022-12-25 13:04:30.981016+00:00,2022-12-25 13:04:30.981016+00:00,2022-12-25 13:04:30.981016+00:00
current_market_yearfrac,0.000,0.000,0.000,0.008,0.008,0.008
